In [2]:
#!pip install PyMuPDF

In [19]:
import glob
import fitz  # PyMuPDF
import re
pdf_path_list=glob.glob("../data/*/*.pdf",
                        recursive=True)
len(pdf_path_list)

57177

In [87]:
import json
from tqdm import tqdm
save_path="output.jsonl"

with open(save_path,"w") as f:
  f.write("")

path=pdf_path_list[14]

for path in tqdm(pdf_path_list):
    filename=path.split("/")[-1].split(".")[0]

    try:
        doc = fitz.open(path)
        text_list=[]
        for page in doc:
            tmp_text = page.get_text("text")
            blocks = page.get_text("blocks")
            #for block in blocks[1:-1]:
            for block in blocks:
                #4つめのblockにテキスト｡ それ以外は座標?
                block_4 = block[4]
                block_4 = re.sub('\n', '', block_4)
                block_4 = re.sub('\u3000', '', block_4)
                if block_4.find("<image:")==0:
                    continue
                if len(block_4) > 5:
                    text_list.append(block_4)

        txt="\n".join(text_list)
        with open(save_path, "a", encoding='utf-8', errors='ignore') as f:
            f.write(json.dumps({"file":filename,"text":txt},ensure_ascii=False)+"\n")

    except:
        continue

  0%|          | 96/57177 [00:06<58:12, 16.34it/s]  